In [1]:

%matplotlib inline
import os
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import SimpleRNN

from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import math
from sklearn.metrics import mean_squared_error
import FinanceDataReader as fdr

Using TensorFlow backend.


In [20]:
#특성 추가 ------------------------------------------
    #이동평균선
def get_MA(df):
    MA_26=df["Close"].rolling(26).mean()
    MA_52=df["Close"].rolling(52).mean()
    df=df.assign(MA_26=MA_26,MA_52=MA_52).dropna()
    
    return df

    
    #스토캐스틱
def get_stochastic(df, n=15, m=5, t=3):
    # n일중 최고가
    ndays_high = df.High.rolling(window=n, min_periods=1).max()
    # n일중 최저가
    ndays_low = df.Low.rolling(window=n, min_periods=1).min()
 
    # Fast%K 계산
    kdj_k = ((df.Close - ndays_low) / (ndays_high - ndays_low))*100
    # Fast%D (=Slow%K) 계산
    kdj_d = kdj_k.ewm(span=m).mean()
    # Slow%D 계산
    kdj_j = kdj_d.ewm(span=t).mean()
 
    # dataframe에 컬럼 추가
    df = df.assign(kdj_k=kdj_k, kdj_d=kdj_d, kdj_j=kdj_j).dropna()
    
    return df
   
    #시간
def get_time(df):
    time=np.linspace(0,10,len(df),endpoint=False).reshape(-1,1)
    df=df.assign(time=time)
    
    return df

In [19]:
KS11 = fdr.DataReader("KS11","2009-01-01","2019-01-01")
KQ11= fdr.DataReader("KQ11","2009-01-01","2019-01-01")
US500 = fdr.DataReader("US500","2009-01-01","2019-01-01")
HSI = fdr.DataReader("HSI","2009-01-01","2019-01-01")
IXIC = fdr.DataReader("IXIC","2009-01-01","2019-01-01")

In [3]:
KS11.shape

(2474, 6)

In [8]:
KQ11.shape

(2475, 6)

In [10]:
HSI.shape

(2474, 6)

In [11]:
IXIC.shape

(2516, 6)

In [17]:
US500.shape

(2516, 6)

In [ ]:
index_dic={}
for x,y in zip([KS11,KQ11,HSI,US500,IXIC],["KS11","KQ11","HSI","US500","IXIC"]):
    a=get_time(get_stochastic(get_MA(x)))
    index_dic[y]=a

In [99]:
#----------------------------------

In [208]:

df=index_dic["KS11"]
MA_26=df["Close"].rolling(26).mean().dropna()
df_before=df["Close"][25:43]
y=np.where(df_before.shift(-1)>MA_26[:18].shift(-1),1,0)

df = df.values
nparr.astype('float32')
scaler = MinMaxScaler(feature_range=(0, 1))
df = scaler.fit_transform(df)
X=df[25:43]


train_size = int(15)
y_train2=y[:train_size]
y_test2=y[train_size:]
X_train2 = X[:train_size]
X_test2 = X[train_size:]

# reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train2, (X_train2.shape[0], 1, X_train2.shape[1]))
X_test = np.reshape(X_test2, (X_test2.shape[0], 1, X_test2.shape[1]))
y_train=y_train2
y_test=y_test2

# simple lstm network learning
model = Sequential()
model.add(LSTM(19, input_shape=(1, 12)))
model.add(Dense(3,activation='sigmoid'))
model.add(Dense(3,activation='sigmoid'))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.fit(X_train, y_train, epochs=10, batch_size=16,validation_split=0.1)

if(model.predict(X_test)[-1]>0.5):
    print("19일날 상승예측")
else:
    print("19일날 하락예측")

Train on 13 samples, validate on 2 samples
Epoch 1/10
13/13 [==============================] - 1s 59ms/step - loss: 0.6973 - acc: 0.0000e+00 - val_loss: 0.6682 - val_acc: 1.0000
Epoch 2/10
13/13 [==============================] - 0s 388us/step - loss: 0.6682 - acc: 1.0000 - val_loss: 0.6400 - val_acc: 1.0000
Epoch 3/10
13/13 [==============================] - 0s 1ms/step - loss: 0.6400 - acc: 1.0000 - val_loss: 0.6126 - val_acc: 1.0000
Epoch 4/10
13/13 [==============================] - 0s 690us/step - loss: 0.6126 - acc: 1.0000 - val_loss: 0.5862 - val_acc: 1.0000
Epoch 5/10
13/13 [==============================] - 0s 460us/step - loss: 0.5862 - acc: 1.0000 - val_loss: 0.5608 - val_acc: 1.0000
Epoch 6/10
13/13 [==============================] - 0s 460us/step - loss: 0.5608 - acc: 1.0000 - val_loss: 0.5362 - val_acc: 1.0000
Epoch 7/10
13/13 [==============================] - 0s 617us/step - loss: 0.5362 - acc: 1.0000 - val_loss: 0.5125 - val_acc: 1.0000
Epoch 8/10
13/13 [==============

In [204]:
len(df)

2423

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices